In [1]:
pwd

'/home/mike/repos/AttentiveWorldModels'

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K
import numpy as np
import os
from utils import preprocess_images

In [2]:
from models import Encoder as buildVision
from models import M as buildMemory

In [3]:
V = buildVision()

In [4]:
M = buildMemory()

In [5]:
M.layers

In [6]:
get_hidden = K.function(M.layers[0].input, M.layers[0].output)

In [7]:
some_z = np.load('./data/z_states/z_state0_1005.npy')

In [8]:
some_a = np.array([1,0,0])

In [9]:
combined = np.concatenate([some_z, some_a], axis=0)

In [10]:
inputs = np.zeros((128, *combined.shape))
inputs[0] = combined

In [11]:
inputs.shape

(128, 35)

In [12]:
M.reset_states()

In [13]:
M.layers[0]

In [14]:
foo = get_hidden(tf.expand_dims(tf.expand_dims(combined, 0), 0))

In [15]:
controller = Sequential([
    Input(shape=(32+256,)),
    Dense(3)
])

In [16]:
ls ./data/z_states

z_state0_0.npy       z_state144_565.npy   z_state5_433.npy
z_state0_1000.npy    z_state144_566.npy   z_state54_33.npy
z_state0_1001.npy    z_state144_567.npy   z_state54_340.npy
z_state0_1002.npy    z_state144_568.npy   z_state54_341.npy
z_state0_1003.npy    z_state144_569.npy   z_state54_342.npy
z_state0_1004.npy    z_state14_456.npy    z_state54_343.npy
z_state0_1005.npy    z_state144_56.npy    z_state54_344.npy
z_state0_1006.npy    z_state144_570.npy   z_state54_345.npy
z_state0_1007.npy    z_state144_571.npy   z_state54_346.npy
z_state0_1008.npy    z_state144_572.npy   z_state54_347.npy
z_state0_1009.npy    z_state144_573.npy   z_state54_348.npy
z_state0_100.npy     z_state144_574.npy   z_state54_349.npy
z_state0_1010.npy    z_state144_575.npy   z_state5_434.npy
z_state0_1011.npy    z_state144_576.npy   z_state54_34.npy
z_state0_1012.npy    z_state144_577.npy   z_state54_350.npy
z_state0_1013.npy    z_state144_578.npy   z_state54_351.npy
z_state0_1014.npy    z_state144_579.npy   z_

z_state10_28.npy     z_state148_268.npy   z_state5_804.npy
z_state102_8.npy     z_state148_269.npy   z_state5_805.npy
z_state102_900.npy   z_state14_826.npy    z_state5_806.npy
z_state102_901.npy   z_state148_26.npy    z_state5_807.npy
z_state102_902.npy   z_state148_270.npy   z_state5_808.npy
z_state102_903.npy   z_state148_271.npy   z_state5_809.npy
z_state102_904.npy   z_state148_272.npy   z_state5_80.npy
z_state102_905.npy   z_state148_273.npy   z_state58_0.npy
z_state102_906.npy   z_state148_274.npy   z_state58_1000.npy
z_state102_907.npy   z_state148_275.npy   z_state58_1001.npy
z_state102_908.npy   z_state148_276.npy   z_state58_1002.npy
z_state102_909.npy   z_state148_277.npy   z_state58_1003.npy
z_state10_290.npy    z_state148_278.npy   z_state58_1004.npy
z_state102_90.npy    z_state148_279.npy   z_state58_1005.npy
z_state102_910.npy   z_state14_827.npy    z_state58_1006.npy
z_state102_911.npy   z_state148_27.npy    z_state58_1007.npy
z_state102_912.npy   z_state148_280.npy   

z_state106_762.npy   z_state16_202.npy    z_state61_939.npy
z_state106_763.npy   z_state16_203.npy    z_state6_193.npy
z_state106_764.npy   z_state16_204.npy    z_state61_93.npy
z_state106_765.npy   z_state16_205.npy    z_state61_940.npy
z_state106_766.npy   z_state16_206.npy    z_state61_941.npy
z_state106_767.npy   z_state16_207.npy    z_state61_942.npy
z_state106_768.npy   z_state16_208.npy    z_state61_943.npy
z_state106_769.npy   z_state16_209.npy    z_state61_944.npy
z_state10_676.npy    z_state1_620.npy     z_state61_945.npy
z_state106_76.npy    z_state16_20.npy     z_state61_946.npy
z_state106_770.npy   z_state16_210.npy    z_state61_947.npy
z_state106_771.npy   z_state16_211.npy    z_state61_948.npy
z_state106_772.npy   z_state16_212.npy    z_state61_949.npy
z_state106_773.npy   z_state16_213.npy    z_state6_194.npy
z_state106_774.npy   z_state16_214.npy    z_state61_94.npy
z_state106_775.npy   z_state16_215.npy    z_state61_950.npy
z_state106_776.npy   z_state16_216.npy    z_

z_state111_332.npy   z_state20_730.npy    z_state66_470.npy
z_state111_333.npy   z_state20_731.npy    z_state66_471.npy
z_state111_334.npy   z_state20_732.npy    z_state66_472.npy
z_state111_335.npy   z_state20_733.npy    z_state66_473.npy
z_state111_336.npy   z_state20_734.npy    z_state66_474.npy
z_state111_337.npy   z_state20_735.npy    z_state66_475.npy
z_state111_338.npy   z_state20_736.npy    z_state66_476.npy
z_state111_339.npy   z_state20_737.npy    z_state66_477.npy
z_state11_133.npy    z_state20_738.npy    z_state66_478.npy
z_state111_33.npy    z_state20_739.npy    z_state66_479.npy
z_state111_340.npy   z_state20_73.npy     z_state6_647.npy
z_state111_341.npy   z_state20_740.npy    z_state66_47.npy
z_state111_342.npy   z_state20_741.npy    z_state66_480.npy
z_state111_343.npy   z_state20_742.npy    z_state66_481.npy
z_state111_344.npy   z_state20_743.npy    z_state66_482.npy
z_state111_345.npy   z_state20_744.npy    z_state66_483.npy
z_state111_346.npy   z_state20_745.npy    

z_state115_522.npy   z_state24_964.npy    z_state70_767.npy
z_state115_523.npy   z_state24_965.npy    z_state70_768.npy
z_state115_524.npy   z_state24_966.npy    z_state70_769.npy
z_state115_525.npy   z_state24_967.npy    z_state70_76.npy
z_state115_526.npy   z_state24_968.npy    z_state70_770.npy
z_state115_527.npy   z_state24_969.npy    z_state70_771.npy
z_state115_528.npy   z_state2_496.npy     z_state70_772.npy
z_state115_529.npy   z_state24_96.npy     z_state70_773.npy
z_state11_552.npy    z_state24_970.npy    z_state70_774.npy
z_state115_52.npy    z_state24_971.npy    z_state70_775.npy
z_state115_530.npy   z_state24_972.npy    z_state70_776.npy
z_state115_531.npy   z_state24_973.npy    z_state70_777.npy
z_state115_532.npy   z_state24_974.npy    z_state70_778.npy
z_state115_533.npy   z_state24_975.npy    z_state70_779.npy
z_state115_534.npy   z_state24_976.npy    z_state70_77.npy
z_state115_535.npy   z_state24_977.npy    z_state70_780.npy
z_state115_536.npy   z_state24_978.npy    

...

z_state12_424.npy    z_state33_76.npy     z_state79_503.npy
z_state124_24.npy    z_state33_770.npy    z_state79_504.npy
z_state124_250.npy   z_state33_771.npy    z_state79_505.npy
z_state124_251.npy   z_state33_772.npy    z_state79_506.npy
z_state124_252.npy   z_state33_773.npy    z_state79_507.npy
z_state124_253.npy   z_state33_774.npy    z_state79_508.npy
z_state124_254.npy   z_state33_775.npy    z_state79_509.npy
z_state124_255.npy   z_state33_776.npy    z_state7_950.npy
z_state124_256.npy   z_state33_777.npy    z_state79_50.npy
z_state124_257.npy   z_state33_778.npy    z_state79_510.npy
z_state124_258.npy   z_state33_779.npy    z_state79_511.npy
z_state124_259.npy   z_state3_377.npy     z_state79_512.npy
z_state12_425.npy    z_state33_77.npy     z_state79_513.npy
z_state124_25.npy    z_state33_780.npy    z_state79_514.npy
z_state124_260.npy   z_state33_781.npy    z_state79_515.npy
z_state124_261.npy   z_state33_782.npy    z_state79_516.npy
z_state124_262.npy   z_state33_783.npy    

z_state128_1021.npy  z_state37_657.npy    z_state83_461.npy
z_state128_1022.npy  z_state37_658.npy    z_state83_462.npy
z_state128_1023.npy  z_state37_659.npy    z_state83_463.npy
z_state128_102.npy   z_state3_765.npy     z_state83_464.npy
z_state128_103.npy   z_state37_65.npy     z_state83_465.npy
z_state128_104.npy   z_state37_660.npy    z_state83_466.npy
z_state128_105.npy   z_state37_661.npy    z_state83_467.npy
z_state128_106.npy   z_state37_662.npy    z_state83_468.npy
z_state128_107.npy   z_state37_663.npy    z_state83_469.npy
z_state128_108.npy   z_state37_664.npy    z_state8_346.npy
z_state128_109.npy   z_state37_665.npy    z_state83_46.npy
z_state12_810.npy    z_state37_666.npy    z_state83_470.npy
z_state128_10.npy    z_state37_667.npy    z_state83_471.npy
z_state128_110.npy   z_state37_668.npy    z_state83_472.npy
z_state128_111.npy   z_state37_669.npy    z_state83_473.npy
z_state128_112.npy   z_state3_766.npy     z_state83_474.npy
z_state128_113.npy   z_state37_66.npy     

z_state131_968.npy   z_state41_627.npy    z_state87_361.npy
z_state131_969.npy   z_state41_628.npy    z_state87_362.npy
z_state13_196.npy    z_state41_629.npy    z_state87_363.npy
z_state131_96.npy    z_state4_162.npy     z_state87_364.npy
z_state131_970.npy   z_state41_62.npy     z_state87_365.npy
z_state131_971.npy   z_state41_630.npy    z_state87_366.npy
z_state131_972.npy   z_state41_631.npy    z_state87_367.npy
z_state131_973.npy   z_state41_632.npy    z_state87_368.npy
z_state131_974.npy   z_state41_633.npy    z_state87_369.npy
z_state131_975.npy   z_state41_634.npy    z_state8_736.npy
z_state131_976.npy   z_state41_635.npy    z_state87_36.npy
z_state131_977.npy   z_state41_636.npy    z_state87_370.npy
z_state131_978.npy   z_state41_637.npy    z_state87_371.npy
z_state131_979.npy   z_state41_638.npy    z_state87_372.npy
z_state13_197.npy    z_state41_639.npy    z_state87_373.npy
z_state131_97.npy    z_state4_163.npy     z_state87_374.npy
z_state131_980.npy   z_state41_63.npy     

z_state135_446.npy   z_state45_138.npy    z_state90_881.npy
z_state135_447.npy   z_state45_139.npy    z_state90_882.npy
z_state135_448.npy   z_state4_513.npy     z_state90_883.npy
z_state135_449.npy   z_state45_13.npy     z_state90_884.npy
z_state13_544.npy    z_state45_140.npy    z_state90_885.npy
z_state135_44.npy    z_state45_141.npy    z_state90_886.npy
z_state135_450.npy   z_state45_142.npy    z_state90_887.npy
z_state135_451.npy   z_state45_143.npy    z_state90_888.npy
z_state135_452.npy   z_state45_144.npy    z_state90_889.npy
z_state135_453.npy   z_state45_145.npy    z_state90_88.npy
z_state135_454.npy   z_state45_146.npy    z_state90_890.npy
z_state135_455.npy   z_state45_147.npy    z_state90_891.npy
z_state135_456.npy   z_state45_148.npy    z_state90_892.npy
z_state135_457.npy   z_state45_149.npy    z_state90_893.npy
z_state135_458.npy   z_state4_514.npy     z_state90_894.npy
z_state135_459.npy   z_state45_14.npy     z_state90_895.npy
z_state13_545.npy    z_state45_150.npy   

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
import gym

In [18]:
env = gym.make("CarRacing-v0")

In [19]:
state = preprocess_images(env.reset(), 1)
env.close()

Track generation: 1255..1575 -> 320-tiles track


In [20]:
state.shape

(1, 64, 64, 3)

In [21]:
def rollout(controller):
    state = preprocess_images(env.reset(), 1)
    
    M.reset_states()
    h = np.zeros(256)
    done = False
    
    while not done:
        _state = np.zeros((128, 64, 64, 3))
        _state[0] = state
        z = V(_state)[0]
        import pdb; pdb.set_trace()
        combined = np.concatenate([z, h], axis=0)
        a = controller.action(combined)
        
        state, reward, done, info = env.step(a)
        state = preprocess_images(state, 1)
        
        cumulative_reward += reward
        
        h = M(tf.expand_dims(tf.expand_dims(np.concatenate([z, a]), 0), 0))[0][0]
    
    return cumulative_reward

In [ ]:
r = rollout(controller)

Track generation: 946..1194 -> 248-tiles track

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

